# Imports

In [ ]:
%%capture

!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa==0.8.1

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm
import torch

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


# Download Dataset

In [ ]:
# !wget https://www.openslr.org/resources/12/train-clean-100.tar.gz
# !wget https://www.openslr.org/resources/12/test-clean.tar.gz
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz

--2023-06-05 18:53:58--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/dev-clean.tar.gz [following]
--2023-06-05 18:53:58--  http://us.openslr.org/resources/12/dev-clean.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: ‘dev-clean.tar.gz’

dev-clean.tar.gz    100%[===================>] 322.27M  29.0MB/s    in 12s     

2023-06-05 18:54:10 (27.2 MB/s) - ‘dev-clean.tar.gz’ saved [337926286/337926286]



In [ ]:
# !tar -xf train-clean-100.tar.gz
# !tar -xf test-clean.tar.gz
!tar -xf /content/dev-clean.tar.gz

# Data preparation

In [ ]:
data = []

# for path in tqdm(Path("/content/LibriSpeech/train-clean-100/").glob("**/**/*.flac")):
for path in tqdm(Path("/content/LibriSpeech/dev-clean/").glob("**/**/*.flac")):
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('/')[-3]
    
    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "info": name,
            "path": path,
            "speaker": label
        })
    except Exception as e:
        # print(str(path), e)
        pass

    # break

2703it [00:06, 395.82it/s]


In [ ]:
df = pd.DataFrame(data)
df.head()

,info,path,speaker
0,422-122949-0015,/content/LibriSpeech/dev-clean/422/122949/422-...,422
1,422-122949-0029,/content/LibriSpeech/dev-clean/422/122949/422-...,422
2,422-122949-0010,/content/LibriSpeech/dev-clean/422/122949/422-...,422
3,422-122949-0013,/content/LibriSpeech/dev-clean/422/122949/422-...,422
4,422-122949-0012,/content/LibriSpeech/dev-clean/422/122949/422-...,422


In [ ]:
# Filter broken and non-existed paths

print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()

Step 0: 2703
Step 1: 2703


<ipython-input-7-5cbd544fec54>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("status", 1)


,info,path,speaker
0,1988-148538-0000,/content/LibriSpeech/dev-clean/1988/148538/198...,1988
1,2803-154320-0000,/content/LibriSpeech/dev-clean/2803/154320/280...,2803
2,2803-161169-0003,/content/LibriSpeech/dev-clean/2803/161169/280...,2803
3,2277-149897-0015,/content/LibriSpeech/dev-clean/2277/149897/227...,2277
4,5895-34622-0018,/content/LibriSpeech/dev-clean/5895/34622/5895...,5895


In [ ]:
#Let's display some random sample of the dataset and run it a couple of times to get a feeling for the audio and the emotional label.
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
path = sample["path"]
label = sample["speaker"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()
speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 77
      Label: 422



In [ ]:
# !mkdir /content/data

In [ ]:
save_path = "/content/drive/MyDrive/Grad project/Data_paths"

# train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["emotion"])

# train_df = train_df.reset_index(drop=True)
# test_df = test_df.reset_index(drop=True)

df.to_csv(f"{save_path}/dev_clean.csv", sep="\t", encoding="utf-8", index=False)


print(df.shape)


(2703, 3)


# Load the Dataset




In [ ]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric


data_files = {
    "data": "/content/drive/MyDrive/Grad project/Data_paths/dev_clean.csv"
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
dataset_dict = dataset["data"]

print(dataset_dict)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating data split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0ff1ad1297f7fcd8/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['info', 'path', 'speaker'],
    num_rows: 2703
})


#HUBERT

In [ ]:
from transformers import AutoProcessor, HubertModel
from datasets import load_dataset
import soundfile as sf
from transformers import Wav2Vec2FeatureExtractor
import torch


processor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")

# processor = AutoProcessor.from_pretrained("facebook/hubert-base-ls960")
model = HubertModel.from_pretrained("facebook/hubert-base-ls960",output_hidden_states=True)
model = model.to(device)


In [ ]:
# create logger
import logging

logname = "/content/drive/MyDrive/Grad project/models_hidden_states/logger.log"
os.makedirs("/content/drive/MyDrive/Grad project/models_hidden_states", exist_ok=True)

logging.basicConfig(filename=logname,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    force=True)

logging.info("init logger")


In [ ]:

model.eval()

try:
  logging.info("----------------New RUN-----------------------")
except:
  print("logger not working")

# iterate over the dataset
for index, utterance in enumerate(tqdm(dataset_dict)):
  # print(index)
  # preprocesses each utterance
  speech, samplerate = sf.read(utterance["path"])
  input_values = processor(speech, return_tensors="pt", sampling_rate = samplerate).input_values.to(device)  # Batch size 1
  # produce the hidden states from Hubert
  with torch.no_grad():
    hidden_states = model(input_values).hidden_states
  #transform into numpy array
  layers = []
  for item in hidden_states:
    layers.append(item.detach().cpu().numpy())
  layers = np.array(layers)
  #save the hidden states
  path = utterance["path"].replace("flac","npy").replace("LibriSpeech", "Hubert_outputs").replace("/content","/content/drive/MyDrive/Grad project/models_hidden_states")
  dir_path = "/".join(path.split("/")[0:-1])
  os.makedirs(dir_path, exist_ok=True)
  np.save(path,layers)
  try:
    speaker = utterance["speaker"]
    file_path = utterance["path"]
    logging.info(f"iteration id:{index}, speaker id:{speaker},  file path:{file_path}")
  except:
    print("logger not working")


100%|██████████| 2703/2703 [1:59:49<00:00,  2.66s/it]


#WAV2VEC2

In [ ]:
 from transformers import  AutoFeatureExtractor, Wav2Vec2Processor, AutoModelForCTC
 from datasets import load_dataset
 import torch
 import soundfile as sf


processor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h",output_hidden_states=True)
model = model.to(device)



Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# create logger
import logging

logname = "/content/drive/MyDrive/Grad project/models_hidden_states/loggerWav2Vec.log"
os.makedirs("/content/drive/MyDrive/Grad project/models_hidden_states", exist_ok=True)

logging.basicConfig(filename=logname,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    force=True)

logging.info("init logger")


In [11]:

model.eval()

try:
  logging.info("----------------Wav2Vec2: New RUN-----------------------")
except:
  print("logger not working")

# iterate over the dataset
for index, utterance in enumerate(tqdm(dataset_dict)):
  # print(index)
  # preprocesses each utterance
  speech, samplerate = sf.read(utterance["path"])
  input_values = processor(speech, return_tensors="pt", sampling_rate = samplerate).input_values.to(device)  # Batch size 1
  # produce the hidden states from Hubert
  with torch.no_grad():
    hidden_states = model(input_values).hidden_states
  #transform into numpy array
  layers = []
  for item in hidden_states:
    layers.append(item.detach().cpu().numpy())
  layers = np.array(layers)
  #save the hidden states
  path = utterance["path"].replace("flac","npy").replace("LibriSpeech", "Wav2Vec2").replace("/content","/content/drive/MyDrive/Grad project/models_hidden_states")
  dir_path = "/".join(path.split("/")[0:-1])
  os.makedirs(dir_path, exist_ok=True)
  np.save(path,layers)
  try:
    speaker = utterance["speaker"]
    file_path = utterance["path"]
    logging.info(f"iteration id:{index}, speaker id:{speaker},  file path:{file_path}")
  except:
    print("logger not working")


100%|██████████| 2703/2703 [2:09:10<00:00,  2.87s/it]


In [ ]:
speech, samplerate = sf.read(dataset_dict[0]["path"])


input_values = processor(speech, return_tensors="pt", sampling_rate = samplerate).input_values  # Batch size 1
hidden_states = model(input_values).logits

In [ ]:
print(hidden_states)

tensor([[[ 20.2408, -42.2271, -41.7718,  ...,  -6.7429,  -5.9352,  -6.4622],
         [ 20.3350, -42.3065, -41.8397,  ...,  -6.7430,  -5.8665,  -6.4150],
         [ 20.4140, -42.2907, -41.8217,  ...,  -6.7265,  -5.8305,  -6.3399],
         ...,
         [ 20.6705, -41.7307, -41.2368,  ...,  -6.7628,  -5.8368,  -6.1953],
         [ 20.5414, -41.7867, -41.2950,  ...,  -6.5007,  -5.5559,  -6.0072],
         [ 20.2161, -42.1685, -41.7207,  ...,  -6.5431,  -5.9054,  -6.2849]]],
       grad_fn=<ViewBackward0>)


In [ ]:
#end of script

In [ ]:
from datasets import load_dataset
dset = load_dataset("/content/LibriSpeech")


In [ ]:
dset.head()

AttributeError: ignored

In [ ]:
dset["test"]["label"]

In [ ]:
from transformers import AutoProcessor, HubertModel
from datasets import load_dataset
import soundfile as sf

processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft")


Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def map_to_array(batch):
    speech, _ = sf.read(batch["audio"]["path"])
    batch["speech"] = speech
    return batch

In [ ]:
dset = dset.map(map_to_array, batch_size=100)

In [ ]:
dset

DatasetDict({
    test: Dataset({
        features: ['audio', 'label', 'speech'],
        num_rows: 2620
    })
})

In [ ]:
# dset["test"]["audio"]["path"]
dset["test"]["audio"][0]

{'path': '/content/LibriSpeech/test-clean/1089/134686/1089-134686-0000.flac',
 'array': array([0.00033569, 0.00030518, 0.00036621, ..., 0.00210571, 0.00210571,
        0.00158691]),
 'sampling_rate': 16000}

In [ ]:
output = sf.read(dset["test"]["audio"][0]["path"])

In [ ]:
output

(array([0.00033569, 0.00030518, 0.00036621, ..., 0.00210571, 0.00210571,
        0.00158691]),
 16000)

In [ ]:
input_values = processor(dset["test"]["speech"][0], return_tensors="pt").input_values  # Batch size 1
hidden_states = model(input_values).last_hidden_state

In [ ]:
from transformers import AutoProcessor, HubertModel
from datasets import load_dataset
import soundfile as sf

processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft")


def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch


ds = load_dataset("librispeech_asr", "clean", split="validation")
ds = ds.map(map_to_array)

input_values = processor(ds["speech"][0], return_tensors="pt").input_values  # Batch size 1
hidden_states = model(input_values).last_hidden_state

In [ ]:
from datasets import load_dataset
# dset = load_dataset("librispeech_asr", 'clean')
dset = load_dataset("librispeech_asr", 'clean', split="train.100", streaming=True)
dset_iter = iter(dset)
while next(dset_iter):
  print(next(dset_iter))

# print(next(iter(dset)))

In [ ]:
!curl -X GET \
     "https://datasets-server.huggingface.co/first-rows?dataset=librispeech_asr&config=all&split=train.clean.100"

TypeError: ignored

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
%%capture

!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa==0.8.1

In [ ]:
mkdir -p /content/data

In [ ]:
# # Download the dataset from 

# !mkdir -p /content/data
# !gdown https://drive.google.com/uc?id=1_IAWexEWpH-ly_JaA5EGfZDp-_3flkN1
# !unzip -q aesdd.zip -d /content/data/
# !mv "/content/data/Acted Emotional Speech Dynamic Database/" "/content/drive/MyDrive/Grad project/"

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

In [ ]:
data = []

for path in tqdm(Path("/content/drive/MyDrive/Grad project/").glob("**/*.wav")):
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('/')[-2]
    
    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "name": name,
            "path": path,
            "emotion": label
        })
    except Exception as e:
        # print(str(path), e)
        pass

    # break

605it [00:20, 29.51it/s]


In [ ]:
df = pd.DataFrame(data)
df.head()

,name,path,emotion
0,f01 (3),/content/drive/MyDrive/Grad project/Acted Emot...,fear
1,f02 (3),/content/drive/MyDrive/Grad project/Acted Emot...,fear
2,f03 (3),/content/drive/MyDrive/Grad project/Acted Emot...,fear
3,f04 (3),/content/drive/MyDrive/Grad project/Acted Emot...,fear
4,f05 (3),/content/drive/MyDrive/Grad project/Acted Emot...,fear


In [ ]:
# Filter broken and non-existed paths

print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()

Step 0: 604
Step 1: 604


<ipython-input-8-5cbd544fec54>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("status", 1)


,name,path,emotion
0,h02 (4),/content/drive/MyDrive/Grad project/Acted Emot...,happiness
1,a16 (1),/content/drive/MyDrive/Grad project/Acted Emot...,anger
2,f18 (5)b,/content/drive/MyDrive/Grad project/Acted Emot...,fear
3,d17 (6),/content/drive/MyDrive/Grad project/Acted Emot...,disgust
4,f02 (2),/content/drive/MyDrive/Grad project/Acted Emot...,fear


In [ ]:
#Let's display some random sample of the dataset and run it a couple of times to get a feeling for the audio and the emotional label.
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
path = sample["path"]
label = sample["emotion"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()
speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 299
      Label: anger



In [ ]:
save_path = "/content/data"

train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["emotion"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(483, 3)
(121, 3)


In [ ]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric


data_files = {
    "train": "/content/data/train.csv", 
    "validation": "/content/data/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cea8daa6e33be702/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['name', 'path', 'emotion'],
    num_rows: 483
})
Dataset({
    features: ['name', 'path', 'emotion'],
    num_rows: 121
})


In [ ]:
# We need to specify the input and output column
input_column = "path"
output_column = "emotion"

In [ ]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 5 classes: ['anger', 'disgust', 'fear', 'happiness', 'sadness']


In order to preprocess the audio into our classification model, we need to set up the relevant HuBERT scale in this case facebook/hubert-base-ls960 by FaceBook HuBERT-Base. To handle the context representations in any audio length we use a merge strategy plan (pooling mode) to concatenate that 3D representations into 2D representations.

There are three merge strategies mean, sum, and max. In this example, we achieved better results on the mean approach. In the following, we need to initiate the config and the feature extractor from the Dimitris model.

In [ ]:
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [ ]:
model_name_or_path = "facebook/hubert-base-ls960"
pooling_mode = "mean"

In [ ]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path,)
target_sampling_rate = feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


# Preprocess Data

So far, we downloaded, loaded, and split the SER dataset into train and test sets. The instantiated our strategy configuration for using context representations in our classification problem SER. Now, we need to extract features from the audio path in context representation tensors and feed them into our classification model to determine the emotion in the speech.

Since the audio file is saved in the `.wav` format, it is easy to use **[Librosa](https://librosa.org/doc/latest/index.html)** or others, but we suppose that the format may be in the `.mp3` format in case of generality. We found that the **[Torchaudio](https://pytorch.org/audio/stable/index.html)** library works best for reading in `.mp3` data.

An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a **map(...)** function accordingly. Also, we need to handle the string labels into integers for our specific classification task in this case, the **single-label classification** you may want to use for your **regression** or even **multi-label classification**.

In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4
)

In [ ]:
idx = 0
print(f"Training input_values: {train_dataset[idx]['input_values']}")
# print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['emotion']}")

In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput
import torch
import torch.nn as nn
from transformers.models.hubert.modeling_hubert import (
    HubertPreTrainedModel,
    HubertModel
)

In [ ]:
hubert = HubertModel(config)

In [ ]:
input_features = [{"input_values": feature["input_values"]} for feature in train_dataset]

In [ ]:
train_dataset[0]["input_values"]

In [ ]:
hubert.eval()
outputs = hubert(
            train_dataset[0]["input_values"],
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
        )

TypeError: ignored

Great, now we've successfully read all the audio files, resampled the audio files to 16kHz, and mapped each audio to the corresponding label.

## Model

Before diving into the training part, we need to build our classification model based on the merge strategy. 

In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [ ]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

# from transformers.models.wav2vec2.modeling_wav2vec2 import (
#     Wav2Vec2PreTrainedModel,
#     Wav2Vec2Model
# )
from transformers.models.hubert.modeling_hubert import (
    HubertPreTrainedModel,
    HubertModel
)


class HubertClassificationHead(nn.Module):
    """Head for hubert classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class HubertForSpeechClassification(HubertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.hubert = HubertModel(config)
        self.classifier = HubertClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.hubert.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.hubert(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2FeatureExtractor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        feature_extractor (:class:`~transformers.Wav2Vec2FeatureExtractor`)
            The feature_extractor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.feature_extractor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(feature_extractor=feature_extractor, padding=True)

Next, the evaluation metric is defined. There are many pre-defined metrics for classification/regression problems, but in this case, we would continue with just **Accuracy** for classification and **MSE** for regression. You can define other metrics on your own.

In [ ]:
is_regression = False

In [ ]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now, we can load the pretrained checkpoint into our classification model with a pooling strategy.

In [ ]:
drive_dir = "/content"
!ls {drive_dir}
output_dir = os.path.join(drive_dir, "ckpts", "hubert-base-greek-ser")
!ls {output_dir}

data  drive  sample_data
ls: cannot access '/content/ckpts/hubert-base-greek-ser': No such file or directory


In [ ]:
import os
import shutil

last_checkpoint = None
checkpoints = []
if os.path.exists(output_dir):
    for subdir in os.scandir(output_dir):
        if subdir.is_dir():
            checkpoints.append(subdir.path)
            
            
if len(checkpoints) > 0:
    checkpoints = list(sorted(checkpoints, key=lambda ckpt: ckpt.split('/')[-1].split('-')[-1], reverse=True))
    model_name_or_path = os.path.join("/content", checkpoints[0].split("/")[-1])
    last_checkpoint = model_name_or_path
    shutil.copytree(checkpoints[0], model_name_or_path)

In [ ]:
print(f"model_name_or_path: {model_name_or_path}")
print(f"last_checkpoint: {last_checkpoint}")

model_name_or_path: facebook/hubert-base-ls960
last_checkpoint: None


In [ ]:
model = HubertForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
    do_train=True,
    do_eval=True,
    do_predict=True
)

ImportError: ignored

For future use we can create our training script, we do it in a simple way. You can add more on you own.

In [ ]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=feature_extractor,
)

### Training

Training will take between 10 and 60 minutes depending on the GPU allocated to this notebook. 

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
if training_args.do_train:
    print(f"last_checkpoint: {last_checkpoint}")
    train_result = trainer.train(resume_from_checkpoint=last_checkpoint)
    trainer.save_model()
    feature_extractor.save_pretrained(training_args.output_dir)
    metrics = train_result.metrics
    metrics["train_samples"] = len(train_dataset)

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

last_checkpoint: None


Step,Training Loss,Validation Loss,Accuracy
10,1.640200,1.593208,0.223140
20,1.565500,1.487835,0.380165
30,1.451700,1.311041,0.404959
40,1.335400,1.469661,0.429752
50,1.373700,1.197774,0.454545
60,1.214400,1.171175,0.487603


The training loss goes down and we can see that the Acurracy on the test set also improves nicely. Because this notebook is just for demonstration purposes, we can stop here.

The resulting model of this notebook has been saved to [m3hrdadfi/hubert-base-greek-speech-emotion-recognition](https://huggingface.co/m3hrdadfi/hubert-base-greek-speech-emotion-recognition)

As a final check, let's load the model and verify that it indeed has learned to recognize the emotion in the speech.

Let's first load the pretrained checkpoint.

## Evaluation

In [ ]:
import librosa
from datasets import load_dataset, load_metric
from sklearn.metrics import classification_report
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [ ]:
test_dataset = load_dataset("csv", data_files={"test": "/content/data/test.csv"}, delimiter="\t")["test"]
test_dataset

Using custom data configuration default-c0949ad448731ab8


Dataset csv downloaded and prepared to /content/cache/csv/default-c0949ad448731ab8/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


Dataset({
    features: ['name', 'path', 'emotion'],
    num_rows: 121
})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [ ]:
model_name_or_path = "m3hrdadfi/hubert-base-greek-speech-emotion-recognition"
config = AutoConfig.from_pretrained(model_name_or_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
model = HubertForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = feature_extractor(batch["speech"], sampling_rate=feature_extractor.sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)

    with torch.no_grad():
        logits = model(input_values).logits 

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

In [ ]:
result = test_dataset.map(predict, batched=True, batch_size=2)

In [ ]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
label_names

['anger', 'disgust', 'fear', 'happiness', 'sadness']

In [ ]:
y_true = [config.label2id[name] for name in result["emotion"]]
y_pred = result["predicted"]

print(y_true[:5])
print(y_pred[:5])

[1, 4, 0, 4, 3]
[1, 4, 0, 4, 3]


In [ ]:
print(classification_report(y_true, y_pred, target_names=label_names))

              precision    recall  f1-score   support

       anger       1.00      0.96      0.98        24
     disgust       0.89      1.00      0.94        24
        fear       1.00      1.00      1.00        24
   happiness       0.95      0.88      0.91        24
     sadness       1.00      1.00      1.00        25

    accuracy                           0.97       121
   macro avg       0.97      0.97      0.97       121
weighted avg       0.97      0.97      0.97       121

